<a href="https://colab.research.google.com/github/adw1029/CSCI_544/blob/main/HW2_CSCI544.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package wordnet to /root/nltk_data...


In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')
google_drive_directory = '/content/drive/My Drive/CSCI_544/hw2'
os.chdir(google_drive_directory)

Mounted at /content/drive


In [5]:
current_directory = os.getcwd()
print("Current Directory:", current_directory)

Current Directory: /content/drive/My Drive/CSCI_544/hw2


#Dataset Generation

In [6]:
file_path = 'amazon_reviews_us_Office_Products_v1_00.tsv'

In [7]:
data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])

<ipython-input-7-c2a96c0343a7>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_path, sep='\t', usecols=['review_body', 'star_rating'])


In [8]:
data = data[data['star_rating'].isin([1, 2, 3, 4, 5])]
data['star_rating'] = data['star_rating'].astype(int)
data = data.dropna(subset=['star_rating', 'review_body'])

In [9]:
group_counts = data.groupby('star_rating').size()
print(group_counts)

star_rating
1     286090
2     129040
3     179866
4     389618
5    1459049
dtype: int64


####get 50k reviews for each rating score

In [10]:
balanced_df = data.groupby('star_rating').sample(n=50000, random_state=42)

In [11]:
group_counts = balanced_df.groupby('star_rating').size()
print(group_counts)

star_rating
1    50000
2    50000
3    50000
4    50000
5    50000
dtype: int64


In [12]:
balanced_df[['star_rating', 'review_body']].isnull().any()

star_rating    False
review_body    False
dtype: bool

###Ternary labels

In [13]:
balanced_df['label'] = balanced_df['star_rating'].apply(lambda x: 1 if x > 3 else (2 if x < 3 else 3))

In [14]:
group_counts = balanced_df.groupby('label').size()
print(group_counts)

label
1    100000
2    100000
3     50000
dtype: int64


#Data Cleaning


#### avg review length and 3 sample reviews before cleaning

In [15]:
avglen_before = balanced_df['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length before data cleaning:", avglen_before)

Average text length before data cleaning: 349.483836


In [16]:
print(balanced_df['review_body'].head(3))

45943      they are cool. good for fidgeting children. Ne...
2008186    Shows too high UV index by far. Have bought an...
336960           Not what I expected it doesn't make copies.
Name: review_body, dtype: object


#### changing reviews to lower case

In [17]:
balanced_df['review_body'] = balanced_df['review_body'].str.lower()

#### remove HTML with beautifulSoup

In [18]:
balanced_df['review_body'] = balanced_df['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())

<ipython-input-18-a80929125028>:1: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  balanced_df['review_body'] = balanced_df['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())
<ipython-input-18-a80929125028>:1: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  balanced_df['review_body'] = balanced_df['review_body'].apply(lambda x: BeautifulSoup(x, "html.parser").get_text())


#### remove URL using regex

In [19]:
balanced_df['review_body'] = balanced_df['review_body'].str.replace(r'http\S+|www.\S+', '', regex=True)

#### remove non-alpha using regex

In [20]:
balanced_df['review_body'] = balanced_df['review_body'].str.replace('[^a-zA-Z]', ' ', regex=True)

#### remove extra spaces using regex

In [21]:
balanced_df['review_body'] = balanced_df['review_body'].str.replace(r'\s+', ' ', regex=True)

#### applying contractions with built-in library

In [22]:
pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 10.9 MB/s eta 0:00:00


In [23]:
import contractions

In [24]:
balanced_df['review_body'] = balanced_df['review_body'].apply(contractions.fix)

#### average review length and 3 sample reviews after cleaning

In [25]:
avglen_after = balanced_df['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length after data cleaning:", avglen_after)

Average text length after data cleaning: 332.593696


In [26]:
print(balanced_df['review_body'].head(3))

45943      they are cool good for fidgeting children need...
2008186    shows too high uv index by far have bought an ...
336960           not what i expected it doesn t make copies 
Name: review_body, dtype: object


# Pre-processing

In [27]:
from nltk.corpus import stopwords

### 3 sample reviews before pre-processing

In [28]:
print(balanced_df['review_body'].head(3))

45943      they are cool good for fidgeting children need...
2008186    shows too high uv index by far have bought an ...
336960           not what i expected it doesn t make copies 
Name: review_body, dtype: object


### average review length before preprocessing

In [29]:
avglen_before = balanced_df['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length before preprocessing:", avglen_before)

Average text length before preprocessing: 332.593696


## remove the stop words

In [30]:
nltk.download('stopwords')

stp_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [31]:
balanced_df['review_body'] = balanced_df['review_body'].apply(lambda x: ' '.join([word for word in str(x).split() if word.lower() not in stp_words]))

## perform lemmatization  

In [32]:
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import wordnet

In [33]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [34]:
lemmatizer = WordNetLemmatizer()

#### applying pos tagging to deal with adj, adv, noun, and verb

In [35]:
get_wordnet_pos = lambda pos: wordnet.ADJ if pos.startswith('J')  else wordnet.VERB if pos.startswith('V') else wordnet.NOUN if pos.startswith('N') else wordnet.ADV if pos.startswith('R') else wordnet.NOUN

In [36]:
!pip install tqdm

In [37]:
from tqdm import tqdm
tqdm.pandas()

In [38]:
balanced_df['review_body'] = balanced_df['review_body'].progress_apply(
    lambda x: ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tag(word_tokenize(x))])
    )

100%|██████████| 250000/250000 [12:31<00:00, 332.50it/s]


In [39]:
print(balanced_df['review_body'].head(3))

45943      cool good fidgeting child need per pencil one ...
2008186    show high uv index far buy electronic device s...
336960                                      expect make copy
Name: review_body, dtype: object


### average review length after  preprocessing

In [40]:
avglen_after = balanced_df['review_body'].apply(lambda x: len(str(x))).mean()

print("Average text length after preprocessing:", avglen_after)

Average text length after preprocessing: 196.63854


#Word Embedding

##Part a

In [67]:
pip install gensim

In [68]:
import gensim.models
from gensim import utils
import gensim.downloader as api
from gensim.models import Word2Vec

In [69]:
model = api.load('word2vec-google-news-300')

####semantic similarities example 1

In [110]:
model_a1 = model.similarity('amazing','fantastic')
print(model_a1)

0.7789871


####semantic similarities example 2

In [111]:
model_a2=model.most_similar(positive=['doctor','woman'], negative=['man'], topn=1)
print(model_a2)

[('gynecologist', 0.7093892097473145)]


##Part b

In [93]:
review_list = [gensim.utils.simple_preprocess(review) for review in balanced_df['review_body']]

In [94]:
model_b = Word2Vec(sentences=review_list, vector_size=300, window = 11, min_count = 10)

In [112]:
model_b1 = model_b.wv.similarity('amazing','fantastic')
print(model_b1)

0.6741172


####semantic similarities example 2

In [114]:
model_b2 = model_b.wv.most_similar(positive=['doctor','woman'], negative=['man'], topn=1)
print(model_b2)

[('medical', 0.6241254210472107)]


#### Comparing part a and part b, for both similarity test, part a model have a better result for both.

#Part3

In [115]:
def get_average_word2vec(doc, model, num_features):
    words = [word for word in doc if word in model.wv.vocab]
    if len(words) == 0:
        return np.zeros(num_features)
    else:
        return np.mean(model.wv[words], axis=0)

###train/test split


In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

NameError: name 'X' is not defined